In [105]:
import pandas as pd
import os
import json

In [106]:
def get_daily_aggregate(participant_id,json_file_name, aggregate_type):
    specific = pd.read_json(f"./PMDATA/{participant_id}/fitbit/{json_file_name}.json")
    if(aggregate_type == "sum"):
        specific.rename(columns = {"value":f"{json_file_name}_sum"}, inplace = True)
        specific = specific.set_index("dateTime").resample("D").sum().reset_index()
    if(aggregate_type == "mean"):
        specific.rename(columns = {"value":f"{json_file_name}_mean"}, inplace = True)
        specific = specific.set_index("dateTime").resample("D").mean().reset_index()
    return specific
    
    
    

In [158]:
def deal_with_sleep_data_json(participant_id):
    sleep_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/sleep.json")
    sleep_df['dateOfSleep'] = pd.to_datetime(sleep_df['dateOfSleep'])
    sleep_df['startTime'] = pd.to_datetime(sleep_df['startTime'])
    sleep_df['endTime'] = pd.to_datetime(sleep_df['endTime'].str.replace('Z', '')) # Handle 'Z' if present
    def extract_sleep_levels(levels_data):
        if isinstance(levels_data, dict) and 'summary' in levels_data:
            summary = levels_data['summary']
            return {
                'deep_sleep_minutes': summary.get('deep', {}).get('minutes'),
                'light_sleep_minutes': summary.get('light', {}).get('minutes'),
                'rem_sleep_minutes': summary.get('rem', {}).get('minutes'),
                'awake_minutes_in_sleep': summary.get('wake', {}).get('minutes')
            }
        return {
            'deep_sleep_minutes': None,
            'light_sleep_minutes': None,
            'rem_sleep_minutes': None,
            'awake_minutes_in_sleep': None
        }
    sleep_levels_extracted = sleep_df['levels'].apply(extract_sleep_levels).apply(pd.Series)
    sleep_df = pd.concat([sleep_df, sleep_levels_extracted], axis=1)
    # Select and rename columns for clarity
    sleep_df_processed = sleep_df[[
        'logId', 'dateOfSleep', 'duration', 'minutesToFallAsleep', 'minutesAsleep',
        'minutesAwake', 'minutesAfterWakeup', 'timeInBed', 'efficiency',
        'deep_sleep_minutes', 'light_sleep_minutes', 'rem_sleep_minutes', 'awake_minutes_in_sleep'
    ]].copy()
    sleep_df_processed.rename(columns={'logId': 'sleep_log_entry_id'}, inplace=True)

    sleep_score_df = pd.read_csv(f"./PMDATA/{participant_id}/fitbit/sleep_score.csv")
    sleep_score_df['timestamp'] = pd.to_datetime(sleep_score_df['timestamp'].str.replace('Z', ''))
    sleep_score_df_processed = sleep_score_df[[
        'sleep_log_entry_id', 'timestamp', 'overall_score', 'composition_score',
        'revitalization_score', 'duration_score', 'restlessness'
    ]].copy()

    sleep_df_processed.drop_duplicates(inplace = True)
    sleep_score_df_processed.drop_duplicates(inplace = True)
    
    combined_sleep_df = pd.merge(
        sleep_df_processed,
        sleep_score_df_processed,
        on='sleep_log_entry_id',
        how='outer'
    )

    combined_sleep_df.rename(columns = {"dateOfSleep":"dateTime"}, inplace = True)

    return combined_sleep_df


    

In [159]:
def preprocess_hr_zones(participant_id):
    hr_zones_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/time_in_heart_rate_zones.json")
    hr_zones_df['dateTime'] = pd.to_datetime(hr_zones_df['dateTime'])

    def extract_hr_zones(value_data):
        if isinstance(value_data, dict) and 'valuesInZones' in value_data:
            zones = value_data['valuesInZones']
            return {
                'time_in_below_default_zone1_minutes': zones.get('BELOW_DEFAULT_ZONE_1', 0.0),
                'time_in_fat_burn_zone_minutes': zones.get('IN_DEFAULT_ZONE_1', 0.0), # Fat Burn Zone 
                'time_in_cardio_zone_minutes': zones.get('IN_DEFAULT_ZONE_2', 0.0),   # Cardio Zone 
                'time_in_peak_zone_minutes': zones.get('IN_DEFAULT_ZONE_3', 0.0)     # Peak Zone 
            }
        # Return None or appropriate default if 'valuesInZones' is missing or not a dict
        return {
            'time_in_below_default_zone1_minutes': None,
            'time_in_fat_burn_zone_minutes': None,
            'time_in_cardio_zone_minutes': None,
            'time_in_peak_zone_minutes': None
        }

    hr_zones_extracted = hr_zones_df['value'].apply(extract_hr_zones).apply(pd.Series)

    hr_zones_df_processed = pd.concat([hr_zones_df['dateTime'], hr_zones_extracted], axis=1)
        
    hr_zones_df_processed['dateTime'] = hr_zones_df_processed['dateTime'].dt.date

    return hr_zones_df_processed

    


In [112]:
def preprocess_heart_rate(participant_id,json_file_name,key_name = "bpm"): 
    heart_rate_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/{json_file_name}.json")
    heart_rate_df["heart_rate_value"] = heart_rate_df["value"].apply(lambda x: x[key_name])
    heart_rate_df = heart_rate_df.drop("value", axis = 1)
    daily_heart_rate = heart_rate_df.set_index("dateTime").resample("D").mean().reset_index()
    daily_heart_rate.rename(columns = {"heart_rate_value":"avg_daily_heart_rate"}, inplace = True)
    daily_heart_rate["dateTime"] = daily_heart_rate["dateTime"].dt.date
    return daily_heart_rate

In [161]:
def preprocess_exercise_data(participant_id):
    exercise_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/exercise.json")

    # Convert 'startTime' to datetime and extract date
    exercise_df['startTime'] = pd.to_datetime(exercise_df['startTime'])
    exercise_df['exercise_date'] = exercise_df['startTime'].dt.date

    selected_exercise_df = exercise_df[[
        "exercise_date",
        "activityName",
        "calories",  # Added 'calories' as per your confirmation
        "duration",
        "steps",
        "distance",
        "averageHeartRate"
    ]].copy()

    daily_exercise_metrics = selected_exercise_df.groupby('exercise_date').agg(
            total_exercise_calories=('calories', 'sum'),
            total_exercise_duration_minutes=('duration', 'sum'),
            total_exercise_steps=('steps', 'sum'),
            total_exercise_distance_km=('distance', 'sum'),
            avg_exercise_heart_rate=('averageHeartRate', 'mean'), # Mean of average HR across sessions
            num_exercise_sessions=('activityName', 'count') # Count of sessions per day
        ).reset_index()
    daily_exercise_metrics.fillna({
        'total_exercise_calories': 0,
        'total_exercise_duration_minutes': 0,
        'total_exercise_steps': 0,
        'total_exercise_distance_km': 0,
    }, inplace=True)

    daily_exercise_metrics.rename(columns = {"exercise_date": "dateTime"}, inplace = True)

    return daily_exercise_metrics

## Heart Rate Data

In [113]:
participant_id= "p07"

In [114]:
path = "./PMDATA/p07/fitbit/"

In [167]:
p07_daily_heart_rate_df = preprocess_heart_rate(participant_id,"heart_rate")
p07_daily_heart_rate_df.head(2)

,dateTime,avg_daily_heart_rate
0,2019-11-06,92.286210
1,2019-11-07,80.636287


In [166]:
resting_heart_rate_df = preprocess_heart_rate(participant_id, "resting_heart_rate",key_name="value")
resting_heart_rate_df.head(2)

,dateTime,avg_daily_heart_rate
0,2019-11-05,0.0
1,2019-11-06,0.0


In [165]:
hr_zones_df = preprocess_hr_zones(participant_id)
hr_zones_df.head(2)

,dateTime,time_in_below_default_zone1_minutes,time_in_fat_burn_zone_minutes,time_in_cardio_zone_minutes,time_in_peak_zone_minutes
0,2019-11-06,302.0,85.0,13.0,0.0
1,2019-11-07,1167.0,115.0,29.0,0.0


## Activity Data

In [163]:
distance_df = get_daily_aggregate(participant_id,"distance","sum")
lightly_active_minutes_df = get_daily_aggregate(participant_id,"lightly_active_minutes","sum")
moderately_active_minutes_df = get_daily_aggregate(participant_id,"moderately_active_minutes","sum")
sedentary_minutes_df = get_daily_aggregate(participant_id,"sedentary_minutes","sum")
very_active_minutes_df = get_daily_aggregate(participant_id,"very_active_minutes","sum")
steps_df = get_daily_aggregate(participant_id,"steps","sum")
very_active_minutes_df .head(2)

,dateTime,very_active_minutes_sum
0,2019-11-05,0
1,2019-11-06,65


In [155]:
hr_zones_df = preprocess_hr_zones(participant_id)
hr_zones_df.head(2)

,dateTime,time_in_below_default_zone1_minutes,time_in_fat_burn_zone_minutes,time_in_cardio_zone_minutes,time_in_peak_zone_minutes
0,2019-11-06,302.0,85.0,13.0,0.0
1,2019-11-07,1167.0,115.0,29.0,0.0


## Sleep Data

In [160]:
sleep_df = deal_with_sleep_data_json(participant_id)
sleep_df.head(2)

,sleep_log_entry_id,dateTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,deep_sleep_minutes,light_sleep_minutes,rem_sleep_minutes,awake_minutes_in_sleep,timestamp,overall_score,composition_score,revitalization_score,duration_score,restlessness
0,24544274440,2019-11-07,31500000,0,443,82,0,525,94,109.0,256.0,78.0,82.0,NaT,NaN,NaN,NaN,NaN,NaN
1,24557669295,2019-11-08,31080000,0,454,64,0,518,98,104.0,241.0,109.0,64.0,2019-11-08 07:02:30,82.0,22.0,20.0,40.0,0.054002


## Exercise Data

In [162]:
exercise_df = preprocess_exercise_data(participant_id)
exercise_df.head(2)

,dateTime,total_exercise_calories,total_exercise_duration_minutes,total_exercise_steps,total_exercise_distance_km,avg_exercise_heart_rate,num_exercise_sessions
0,2019-11-06,629,4912000,6392.0,0.0,113.0,3
1,2019-11-07,948,6345000,9350.0,0.0,119.0,2


# SRPE

In [218]:
def preprocess_wellness_csv(participant_id):
    wellness = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/wellness.csv")
    wellness["effective_time_frame"] = pd.to_datetime(wellness["effective_time_frame"])
    wellness["effective_time_frame"] = wellness["effective_time_frame"].dt.date
    wellness.rename(columns = {"effective_time_frame":"dateTime"}, inplace = True)
    
    wellness[wellness["dateTime"].duplicated()]
    wellness["dateTime"].drop_duplicates(inplace = True)
    wellness.drop("soreness_area", axis = 1, inplace = True)

    return wellness
wellness_df = preprocess_wellness_csv(participant_id)
wellness_df.head(2)

,dateTime,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress
0,2019-11-06,3,3,2,9,4,3,5
1,2019-11-07,4,3,6,9,4,3,4


In [225]:
def preprocess_srpe_data(participant_id):
    srpe_df = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/srpe.csv")

    # 1. Convert 'end_date_time' to datetime and extract date
    srpe_df['end_date_time'] = pd.to_datetime(srpe_df['end_date_time'].str.replace('Z', ''))
    srpe_df['srpe_date'] = srpe_df['end_date_time'].dt.date

    # 2. Calculate Session Training Load (sRPE)
    srpe_df['perceived_exertion'] = pd.to_numeric(srpe_df['perceived_exertion'], errors='coerce')
    srpe_df['duration_min'] = pd.to_numeric(srpe_df['duration_min'], errors='coerce')
    srpe_df['session_load_srpe'] = srpe_df['perceived_exertion'] * srpe_df['duration_min']

    # 3. Handle 'activity_names' (it's a string representation of a list, so convert it)
    import ast
    srpe_df['activity_names_list'] = srpe_df['activity_names'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

    # 4. Aggregate to Daily Level
    daily_srpe_metrics = srpe_df.groupby('srpe_date').agg(
        total_daily_training_load_srpe=('session_load_srpe', 'sum'),
        total_daily_duration_min=('duration_min', 'sum'),
        avg_daily_perceived_exertion=('perceived_exertion', 'mean'),
        num_daily_srpe_sessions=('end_date_time', 'count'), 
        unique_daily_activity_names=('activity_names_list', lambda x: list(set([item for sublist in x for item in sublist])))
    ).reset_index()

    # Rename date column for merging consistency
    daily_srpe_metrics.rename(columns={'srpe_date': 'dateTime'}, inplace=True)

    # Fill NaNs for sum-based metrics with 0 where appropriate (if no training on a day)
    daily_srpe_metrics.fillna({
        'total_daily_training_load_srpe': 0,
        'total_daily_duration_min': 0,
        'avg_daily_perceived_exertion': 0 # Or NaN if you prefer to distinguish no training vs 0 exertion
    }, inplace=True)

    return daily_srpe_metrics

In [243]:
def preprocess_injury_data(participant_id):
    injury_df = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/injury.csv")

    # Convert 'effective_time_frame' to datetime and extract date
    injury_df['effective_time_frame'] = pd.to_datetime(injury_df['effective_time_frame'])
    injury_df['injury_date'] = injury_df['effective_time_frame'].dt.date

    # Process 'injuries' column (which contains dictionaries)
    # Convert string representation of dict to actual dict for empty check
    injury_df['injuries_dict'] = injury_df['injuries'].apply(lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x)

    # Flag if any injury was reported for the entry
    injury_df['has_injury_reported_session'] = injury_df['injuries_dict'].apply(lambda x: 1 if x else 0)

    # Count the number of distinct injury areas reported in that session
    injury_df['num_injury_areas_session'] = injury_df['injuries_dict'].apply(lambda x: len(x) if x else 0)

    # Optional: Extract severity and specific locations if needed for detailed analysis
    # For simplicity, we'll stick to a count and a general flag for the dashboard.
    # If a specific injury (e.g., 'right_foot') is important, you can add flags:
    # injury_df['has_right_foot_injury'] = injury_df['injuries_dict'].apply(lambda x: 1 if 'right_foot' in x else 0)

    # Aggregate to daily level
    # Since it's weekly, and we just want to know if there was an injury report on a day
    daily_injury_metrics = injury_df.groupby('injury_date').agg(
        has_injury_reported=('has_injury_reported_session', 'max'), # Use max to get 1 if any injury was reported that day
        num_unique_injury_areas_daily=('num_injury_areas_session', 'max') # Max number of areas reported on that day
    ).reset_index()

    # Rename the date column for merging consistency
    daily_injury_metrics.rename(columns={'injury_date': 'dateTime'}, inplace=True)

    # Ensure has_injury_reported is int (0 or 1)
    daily_injury_metrics['has_injury_reported'] = daily_injury_metrics['has_injury_reported'].astype(int)

    return daily_injury_metrics

In [247]:
def preprocess_reporting_data(participant_id):
    reporting_df = pd.read_csv(f"./PMDATA/{participant_id}/googledocs/reporting.csv")

    # 1. Convert 'date' and 'timestamp' columns
    # 'date' will be our primary key for daily aggregation
    reporting_df['date'] = pd.to_datetime(reporting_df['date'], format='%d/%m/%Y')
    reporting_df['common_date_column'] = reporting_df['date'].dt.date

    # Convert 'timestamp' for potential ordering in case of duplicates, but not strictly needed for daily agg if 'date' is used.
    # reporting_df['timestamp'] = pd.to_datetime(reporting_df['timestamp'], format='%d/%m/%Y %H:%M:%S')

    # 2. Process 'meals' column to create boolean flags for each meal type
    meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Evening']
    for meal in meal_types:
        # Create a boolean column for each meal type based on its presence in the 'meals' string
        reporting_df[f'had_{meal.lower()}'] = reporting_df['meals'].str.contains(meal, case=False, na=False)

    # 3. Process 'alcohol_consumed' column
    reporting_df['alcohol_consumed_flag'] = reporting_df['alcohol_consumed'].apply(lambda x: 1 if x == 'Yes' else 0)

    # 4. Handle duplicate dates by aggregating to a daily level
    # For numerical columns like 'weight' and 'glasses_of_fluid',
    # if multiple reports exist for a day:
    # - 'weight': take the last reported weight for the day (more likely to be current).
    # - 'glasses_of_fluid': sum all fluid intakes reported for the day.
    # For boolean meal flags and alcohol flag: use 'max' to get 1 if reported at least once.

    # Define aggregation dictionary
    agg_dict = {
        'weight': 'last', # Take the last weight reported for the day
        'glasses_of_fluid': 'sum', # Sum all fluid intake for the day
        'alcohol_consumed_flag': 'max', # 1 if alcohol was consumed at least once, 0 otherwise
    }
    # Add meal flags to aggregation dict using 'max'
    for meal in meal_types:
        agg_dict[f'had_{meal.lower()}'] = 'max'

    daily_reporting_metrics = reporting_df.groupby('common_date_column').agg(
        **agg_dict
    ).reset_index()

    # 5. Handle NaNs in 'weight' after daily aggregation
    # Weight is a time-series, so interpolation or ffill/bfill is better than dropping.
    # We will interpolate linearly to fill gaps, then ffill/bfill for edges if needed.
    daily_reporting_metrics['weight'] = daily_reporting_metrics['weight'].interpolate(method='linear', limit_direction='both', limit_area='inside')
    daily_reporting_metrics['weight'] = daily_reporting_metrics['weight'].fillna(method='ffill').fillna(method='bfill') # Fill remaining NaNs at edges

    # Ensure boolean/int columns are correct type
    for meal in meal_types:
        daily_reporting_metrics[f'had_{meal.lower()}'] = daily_reporting_metrics[f'had_{meal.lower()}'].astype(int)
    daily_reporting_metrics['alcohol_consumed_flag'] = daily_reporting_metrics['alcohol_consumed_flag'].astype(int)

    return daily_reporting_metrics

In [244]:
preprocess_srpe_data(participant_id).head(2)

,dateTime,total_daily_training_load_srpe,total_daily_duration_min,avg_daily_perceived_exertion,num_daily_srpe_sessions,unique_daily_activity_names
0,2019-11-06,120,40,3.0,1,"[individual, endurance]"
1,2019-11-07,350,70,5.0,1,"[individual, endurance]"


In [246]:
injury_df = preprocess_injury_data(participant_id)
injury_df.head(2)

,dateTime,has_injury_reported,num_unique_injury_areas_daily
0,2019-11-11,0,0
1,2019-11-18,0,0


In [248]:
preprocess_reporting_data(participant_id)

TypeError: Must provide 'func' or tuples of '(column, aggfunc).